# Menggunakan BigQuery ML untuk memprediksi hasil bola basket

Tutorial ini menggunakan BigQuery ML untuk memprediksi upaya sasaran lapangan tiga poin dalam bola basket. BigQuery ML memungkinkan pengguna untuk membuat dan menjalankan model pembelajaran mesin di BigQuery menggunakan kueri SQL. Tujuannya adalah untuk mendemokratisasikan pembelajaran mesin dengan memungkinkan praktisi SQL untuk membangun model menggunakan alat yang ada dan untuk meningkatkan kecepatan pengembangan dengan menghilangkan kebutuhan akan pergerakan data.

* Datasets : https://bigquery.cloud.google.com/dataset/bigquery-public-data:ncaa_basketball

## Tujuan

* Membuat model Regresi linier menggunakan CREATE MODEL
* Menggunakan fungsi ML.EVALUATE untuk mengevaluasi model ML
* Menggunakan fungsi ML.PREDICT untuk membuat prediksi menggunakan model ML

## 1. Import library

Mengimpor pustaka klien BigQuery Python dan menginisialisasi klien. Klien BigQuery digunakan untuk mengirim dan menerima pesan dari BigQuery API.

In [1]:
from google.cloud import bigquery

# Menggunakan kredensial Service Account
# client = bigquery.Client.from_service_account_json('key.json')
# Jika tidak mendifinisikan kredensial ke dalam variabel client
# maka library client akan mencari kredensial dari project yang terhubung sekarang
client = bigquery.Client()

## 2. Membuat dataset

In [ ]:
dataset = bigquery.Dataset(client.dataset('bqml_tutorial'))
dataset.location = 'US'
client.create_dataset(dataset)

## 3. Generate your training and feature data

### Generate the input features (feature engineering)

### Generate your training data

## 4. Membuat model

Selanjutnya, Anda membuat model regresi linier. Model ini digunakan untuk memprediksi upaya sasaran lapangan tiga poin gabungan berdasarkan statistik game sebelumnya. Permintaan SQL standar menggunakan pernyataan `` CREATE MODEL `` untuk membuat model. Meskipun pernyataan `` CREATE MODEL `` dapat membuat dan melatih model Anda, dalam tutorial ini, Anda melatih model secara terpisah.

In [3]:
%load_ext google.cloud.bigquery

In [4]:
%%bigquery
CREATE OR REPLACE MODEL bqml_tutorial.ncaa_model OPTIONS(
  model_type='linear_reg', max_iteration=50) AS
SELECT
  * EXCEPT(
    game_id, season, scheduled_date,
    total_three_points_made,
    total_three_points_att),
  total_three_points_att as label
FROM
  bqml_tutorial.wide_games
WHERE
  # remove the game to predict
  game_id != 'f1063e80-23c7-486b-9a5e-faa52beb2d83'

""


## 5. Dapatkan statistik pelatihan

In [5]:
%%bigquery
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `bqml_tutorial.ncaa_model`)

,training_run,iteration,loss,eval_loss,duration_ms,learning_rate
0,0,8,54.285957,125.467233,8515,0.05
1,0,7,64.075624,125.759724,8886,0.05
2,0,6,77.713638,151.677391,8981,0.05
3,0,5,96.864951,154.259379,9236,0.05
4,0,4,124.012543,202.382607,12472,0.05
5,0,3,162.991140,212.495831,11464,0.05
6,0,2,220.158601,305.039517,10190,0.05
7,0,1,308.408170,341.092925,12136,0.05
8,0,0,523.584072,609.183833,6435,0.20


## 6. Evaluasi model

In [6]:
%%bigquery
WITH eval_table AS (
SELECT
  *,
  total_three_points_att AS label
FROM
  `bqml_tutorial.wide_games` )
SELECT
  *
FROM
  ML.EVALUATE(MODEL `bqml_tutorial.ncaa_model`,
    TABLE eval_table)

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,6.423198,68.865911,0.066344,5.41769,0.090511,0.423828


## 7. Gunakan model untuk memprediksi hasil

Sekarang setelah Anda mengevaluasi model Anda, langkah berikutnya adalah menggunakan fungsi ML.PREDICT untuk memprediksi upaya total tiga poin tujuan lapangan dalam pertandingan final NCAA 2018: Michigan versus Villanova.

In [7]:
%%bigquery
  WITH game_to_predict AS (
  SELECT
    *
  FROM
    `bqml_tutorial.wide_games`
  WHERE
    game_id='f1063e80-23c7-486b-9a5e-faa52beb2d83' )
SELECT
  truth.game_id AS game_id,
  total_three_points_att,
  predicted_total_three_points_att
FROM (
  SELECT
    game_id,
    predicted_label AS predicted_total_three_points_att
  FROM
    ML.PREDICT(MODEL `bqml_tutorial.ncaa_model`,
      table game_to_predict) ) AS predict
JOIN (
  SELECT
    game_id,
    total_three_points_att AS total_three_points_att
  FROM
    game_to_predict) AS truth
ON
  predict.game_id = truth.game_id

,game_id,total_three_points_att,predicted_total_three_points_att
0,f1063e80-23c7-486b-9a5e-faa52beb2d83,50,41.192128


Nilai `` total_three_points_att `` adalah jumlah sebenarnya tujuan lapangan yang terjadi di pertandingan terakhir - 50. Prediksi model adalah 43,41.